In [6]:
import sys
sys.executable

'/usr/local/opt/python@2/bin/python2.7'

In [7]:
import os
import numpy as np
import cv2
import tqdm
from tqdm import tqdm
from sklearn.model_selection import train_test_split

TRAIN_IMAGE_DIR = '../input/train/images/'
TRAIN_MASK_DIR = '../input/train/masks/'
TEST_IMAGE_DIR = '../input/test/images/'

train_fns = os.listdir(TRAIN_IMAGE_DIR)

X = [np.array(cv2.imread(TRAIN_IMAGE_DIR + p, cv2.IMREAD_GRAYSCALE), dtype=np.uint8) for p in tqdm(train_fns)]
X = np.array(X)/255
X = np.expand_dims(X,axis=3)

y = [np.array(cv2.imread(TRAIN_MASK_DIR + p, cv2.IMREAD_GRAYSCALE), dtype=np.uint8) for p in tqdm(train_fns)]
y = np.array(y)/255
y = np.expand_dims(y,axis=3)

X_train, X_valid, y_train, y_valid = train_test_split(X,y, random_state=23, test_size = 0.2)

from keras.models import Model
from keras.layers import Conv2D, Input, Concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
import h5py

def conv_block(num_layers,inp,units,kernel):
    x = inp
    for l in range(num_layers):
        x = Conv2D(units, kernel_size=kernel, padding='SAME',activation='relu')(x)
    return x


inp = Input(shape=(101,101,1))
cnn1 = conv_block(2,inp,32,3)
cnn2 = conv_block(2,inp,24,5)
cnn3 = conv_block(2,inp,16,7)
concat = Concatenate()([cnn1,cnn2,cnn3])
d1 = Conv2D(16,1, activation='relu')(concat)
out = Conv2D(1,1, activation='sigmoid')(d1)

model = Model(inputs = inp, outputs = out)
model.summary()
model.compile(optimizer='adam',loss='binary_crossentropy')

early_stop = EarlyStopping(patience=5)
check_point = ModelCheckpoint('model.hdf5',save_best_only=True)
model.fit(X_train,y_train, epochs=50, validation_data=(X_valid,y_valid), callbacks=[early_stop,check_point],batch_size=128)





100%|██████████| 4000/4000 [00:01<00:00, 2430.87it/s]


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 101, 101, 1)  0                                            
__________________________________________________________________________________________________
conv2d_29 (Conv2D)              (None, 101, 101, 32) 320         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_31 (Conv2D)              (None, 101, 101, 24) 624         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_33 (Conv2D)              (None, 101, 101, 16) 800         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_30 

In [8]:

test_fns = os.listdir(TEST_IMAGE_DIR)
X_test = [np.array(cv2.imread(TEST_IMAGE_DIR + p, cv2.IMREAD_GRAYSCALE), dtype=np.uint8) for p in tqdm(test_fns)]
X_test = np.array(X_test)/255
X_test = np.expand_dims(X_test,axis=3)

pred = model.predict(X_test, verbose = True)


def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

pred_dict = {fn[:-4]:RLenc(np.round(pred[i,:,:,0])) for i,fn in tqdm(enumerate(test_fns))}


import pandas as pd



sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('submission.csv')

100%|██████████| 18000/18000 [00:19<00:00, 930.98it/s]


18000/18000 [==============================] - 944s 52ms/step


18000it [05:50, 51.37it/s]


In [21]:
test_fns = os.listdir(TEST_IMAGE_DIR)
X_test1 = [np.array(cv2.imread(TEST_IMAGE_DIR + p, cv2.IMREAD_GRAYSCALE), dtype=np.uint8) for p in tqdm(test_fns)]
X_test2 = np.array(X_test1)/255
X_test3 = np.expand_dims(X_test2,axis=3)



100%|██████████| 18000/18000 [00:19<00:00, 904.57it/s]


In [20]:
X_test3

array([[[[0.43921569],
         [0.49411765],
         [0.57254902],
         ...,
         [0.61568627],
         [0.73333333],
         [0.7254902 ]],

        [[0.45490196],
         [0.50980392],
         [0.57647059],
         ...,
         [0.61176471],
         [0.72941176],
         [0.70196078]],

        [[0.46666667],
         [0.52941176],
         [0.57254902],
         ...,
         [0.62745098],
         [0.7254902 ],
         [0.6745098 ]],

        ...,

        [[0.38039216],
         [0.44313725],
         [0.49019608],
         ...,
         [0.24313725],
         [0.30980392],
         [0.33333333]],

        [[0.39215686],
         [0.45098039],
         [0.48235294],
         ...,
         [0.33333333],
         [0.38823529],
         [0.41176471]],

        [[0.40784314],
         [0.45490196],
         [0.46666667],
         ...,
         [0.42745098],
         [0.47058824],
         [0.49803922]]],


       [[[0.34117647],
         [0.34117647],
         [0.34